In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stat

#Francis: codes
#https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
#%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, IntProgress, HBox, HTML, Text, VBox
import pandas as pd
from datetime import date, timedelta
import datetime
from IPython.display import clear_output
import requests
import threading
import time
import folium
import folium.plugins as plugins

In [ ]:
widgets.HTML(value='<table width="100%"><tr><td align="center"><h1>Visualizations and Simulations for COVID-Related Analytics</h1></td></tr></table><hr/>')

## Model overview

![Test](./logo/SIR_Graph.png)

In [ ]:
from scipy.integrate import odeint

class SeairOde:

    def __init__(self, p=0.995, N_0=67*10**6, R_0=4*10**6, Inf_0 = 20*10**4, ICU_0=4*10**3, number_points_per_day = 500):
        try:
            ######################
            ## Time discretisation
            ######################
            self.number_points_per_day = number_points_per_day

            ######################
            # Parameters
            ######################
            self.parameters = {
                'p': p,             # Proportion of people with mild symptoms
                'Reprod_0': 3.3,    # basic reproduction number
                'eps': 1 / 3.7,     # Rate of passage from E to A
                'sigma': 1 / 1.5,    # Rate of passage from A to I
                'gamma_m': 1 / 2.3,  # Recovery rate for mild symptoms
                'gamma_s': 1 / 17.,  # Recovery rate for severe symptoms
                'alpha': 1 / 11.7,
                'eta': 1 / 7.,


            }

            ######################
            # Initial conditions
            ######################
            var = 1/(1+self.parameters['eps']*((1/self.parameters['sigma'])+(1/self.parameters['gamma_m'])))
            E_0 = var*Inf_0
            A_0 = (self.parameters['eps']/self.parameters['sigma'])*E_0
            I_0 = (self.parameters['eps']/self.parameters['gamma_m'])*E_0
            S_0 = N_0 - (E_0 + A_0 + I_0 + ICU_0 + R_0)
            self.initial_conditions = {'N_0': N_0,  # Total number of individuals at time 0 # Todo : comments okay ?
                                       'R_0': R_0,  # Total number of immune at time 0
                                       'Inf_0': Inf_0, # Total number of infected E + A + I at time 0
                                       'I_0': I_0,  # Total number of asymptomatic  at time 0
                                       'S_0': S_0,  # Total number of susceptibles at time 0
                                       'E_0': E_0,  # Total number of infected at time 0
                                       'A_0': A_0,   # Total number of latent at time 0
                                       'ICU_0': ICU_0
                                       }

            # Finalize parameters
            self.parameters['beta'] = (self.parameters['Reprod_0'] / self.initial_conditions['N_0']) / \
                                      (self.parameters['p']*(1/self.parameters['gamma_m'] + 1/self.parameters['sigma']) +
                                       (1-self.parameters['p'])*(1/self.parameters['eta'] + 1/self.parameters['sigma']) )


            ######################
            # Computing initial conditions in each category as if labeling were perfect
            ######################
            self.initial_conditions_category = {
                # S
                'S_m_0': self.parameters['p'] * self.initial_conditions['S_0'],
                'S_s_0': (1 - self.parameters['p']) * self.initial_conditions['S_0'],
                'S_mr_0':self.parameters['p'] * self.initial_conditions['S_0'],
                'S_mc_0': 0,
                'S_sc_0': (1 - self.parameters['p']) * self.initial_conditions['S_0'],
                'S_sr_0': 0,
                # A
                'A_m_0': self.parameters['p'] * self.initial_conditions['A_0'],
                'A_s_0': (1 - self.parameters['p']) * self.initial_conditions['A_0'],
                'A_mr_0': self.parameters['p'] * self.initial_conditions['A_0'],
                'A_mc_0': 0,
                'A_sc_0': (1 - self.parameters['p']) * self.initial_conditions['A_0'],
                'A_sr_0': 0,
                # E
                'E_m_0': self.parameters['p'] * self.initial_conditions['E_0'],
                'E_s_0': (1 - self.parameters['p']) * self.initial_conditions['E_0'],
                'E_mr_0': self.parameters['p'] * self.initial_conditions['E_0'],
                'E_mc_0': 0,
                'E_sc_0': (1 - self.parameters['p']) * self.initial_conditions['E_0'],
                'E_sr_0': 0,
                # I
                'I_m_0': self.parameters['p'] * self.initial_conditions['I_0'],
                'I_s_0': (1 - self.parameters['p']) * self.initial_conditions['I_0'],
                'I_mr_0': self.parameters['p'] * self.initial_conditions['I_0'],
                'I_mc_0': 0,
                'I_sc_0': (1 - self.parameters['p']) * self.initial_conditions['I_0'],
                'I_sr_0': 0,
                # ICU
                'ICU_c_0': self.initial_conditions['ICU_0'],
                'ICU_r_0': 0,
                # R
                'R_m_0': self.parameters['p'] * self.initial_conditions['R_0'],
                'R_s_0': (1 - self.parameters['p']) * self.initial_conditions['R_0'],
                'R_mr_0': self.parameters['p'] * self.initial_conditions['R_0'],
                'R_mc_0': 0,
                'R_sc_0': (1 - self.parameters['p']) * self.initial_conditions['R_0'],
                'R_sr_0': 0,
                # D
                'D_0' : 0
            }

            ######################
            # Initial condition without any misclassification
            ######################
            self.y_0 = np.array([self.initial_conditions_category['S_mr_0'],
                                 self.initial_conditions_category['E_mr_0'],
                                 self.initial_conditions_category['A_mr_0'],
                                 self.initial_conditions_category['I_mr_0'],
                                 self.initial_conditions_category['R_mr_0'],
                                 self.initial_conditions_category['S_mc_0'],
                                 self.initial_conditions_category['E_mc_0'],
                                 self.initial_conditions_category['A_mc_0'],
                                 self.initial_conditions_category['I_mc_0'],
                                 self.initial_conditions_category['R_mc_0'],
                                 self.initial_conditions_category['S_sc_0'],
                                 self.initial_conditions_category['E_sc_0'],
                                 self.initial_conditions_category['A_sc_0'],
                                 self.initial_conditions_category['I_sc_0'],
                                 self.initial_conditions_category['ICU_c_0'],
                                 self.initial_conditions_category['R_sc_0'],
                                 self.initial_conditions_category['S_sr_0'],
                                 self.initial_conditions_category['E_sr_0'],
                                 self.initial_conditions_category['A_sr_0'],
                                 self.initial_conditions_category['I_sr_0'],
                                 self.initial_conditions_category['ICU_r_0'],
                                 self.initial_conditions_category['R_sr_0'],
                                 self.initial_conditions_category['D_0']])
        except Exception as e:
            print('0')

    def progressive_release(self, T, q, delta_r, delta_c):
        try:
            n = T.size
            t_tot = []
            sol_tot = []
            t_temp = np.linspace(0, T[0], T[0] * self.number_points_per_day)
            ## Update initial condition according to q[0]
            real_y_0 = SeairOde.shuffle_qs(q[0], self.y_0)
            sol_temp = self.prop_ODE(delta_r, delta_c, real_y_0, T[0], T[0] * self.number_points_per_day)
            sol_tot = sol_temp
            t_tot = t_temp
            for i in range(1, n):
                y_temp = SeairOde.shuffle_qs(q[i], sol_temp[-1, :])
                sol_temp = self.prop_ODE(delta_r, delta_c, y_temp, T[i] - T[i - 1],
                                    (T[i] - T[i - 1]) * self.number_points_per_day)
                sol_tot = np.concatenate([sol_tot[:-1], sol_temp])
                t_temp = np.linspace(T[i - 1], T[i], (T[i] - T[i - 1]) * self.number_points_per_day)
                t_tot = np.append(t_tot[:-1], t_temp)
            return t_tot, sol_tot
        except Exception as e:
            print('1')

    @staticmethod
    def shuffle_qs(new_q, old_y):
        try:
            new_q_FP = new_q[0]
            new_q_FN = new_q[1]
            new_S_mr = (1 - new_q_FP) * (old_y[SeairOde.get_y_order().index("S_mr")] + old_y[SeairOde.get_y_order().index("S_mc")])
            new_E_mr = (1 - new_q_FP) * (old_y[SeairOde.get_y_order().index("E_mr")] + old_y[SeairOde.get_y_order().index("E_mc")])
            new_A_mr = (1 - new_q_FP) * (old_y[SeairOde.get_y_order().index("A_mr")] + old_y[SeairOde.get_y_order().index("A_mc")])
            new_I_mr = (1 - new_q_FP) * (old_y[SeairOde.get_y_order().index("I_mr")] + old_y[SeairOde.get_y_order().index("I_mc")])
            new_R_mr = (1 - new_q_FP) * (old_y[SeairOde.get_y_order().index("R_mr")] + old_y[SeairOde.get_y_order().index("R_mc")])
            new_S_mc = new_q_FP * (old_y[SeairOde.get_y_order().index("S_mr")] + old_y[SeairOde.get_y_order().index("S_mc")])
            new_E_mc = new_q_FP * (old_y[SeairOde.get_y_order().index("E_mr")] + old_y[SeairOde.get_y_order().index("E_mc")])
            new_A_mc = new_q_FP * (old_y[SeairOde.get_y_order().index("A_mr")] + old_y[SeairOde.get_y_order().index("A_mc")])
            new_I_mc = new_q_FP * (old_y[SeairOde.get_y_order().index("I_mr")] + old_y[SeairOde.get_y_order().index("I_mc")])
            new_R_mc = new_q_FP * (old_y[SeairOde.get_y_order().index("R_mr")] + old_y[SeairOde.get_y_order().index("R_mc")])
            new_S_sc = (1 - new_q_FN) * (old_y[SeairOde.get_y_order().index("S_sc")] + old_y[SeairOde.get_y_order().index("S_sr")])
            new_E_sc = (1 - new_q_FN) * (old_y[SeairOde.get_y_order().index("E_sc")] + old_y[SeairOde.get_y_order().index("E_sr")])
            new_A_sc = (1 - new_q_FN) * (old_y[SeairOde.get_y_order().index("A_sc")] + old_y[SeairOde.get_y_order().index("A_sr")])
            new_I_sc = (1 - new_q_FN) * (old_y[SeairOde.get_y_order().index("I_sc")] + old_y[SeairOde.get_y_order().index("I_sr")])
            new_ICU_c = (1 - new_q_FN) * (old_y[SeairOde.get_y_order().index("ICU_c")] + old_y[SeairOde.get_y_order().index("ICU_r")])
            new_R_sc = (1 - new_q_FN) * (old_y[SeairOde.get_y_order().index("R_sc")] + old_y[SeairOde.get_y_order().index("R_sr")])
            new_S_sr = new_q_FN * (old_y[SeairOde.get_y_order().index("S_sc")] + old_y[SeairOde.get_y_order().index("S_sr")])
            new_E_sr = new_q_FN * (old_y[SeairOde.get_y_order().index("E_sc")] + old_y[SeairOde.get_y_order().index("E_sr")])
            new_A_sr = new_q_FN * (old_y[SeairOde.get_y_order().index("A_sc")] + old_y[SeairOde.get_y_order().index("A_sr")])
            new_I_sr = new_q_FN * (old_y[SeairOde.get_y_order().index("I_sc")] + old_y[SeairOde.get_y_order().index("I_sr")])
            new_ICU_r = new_q_FN * (old_y[SeairOde.get_y_order().index("ICU_c")] + old_y[SeairOde.get_y_order().index("ICU_r")])
            new_R_sr = new_q_FN * (old_y[SeairOde.get_y_order().index("R_sc")] + old_y[SeairOde.get_y_order().index("R_sr")])
            new_D = old_y[SeairOde.get_y_order().index("D")]
            return np.array([
                new_S_mr,
                new_E_mr,
                new_A_mr,
                new_I_mr,
                new_R_mr,
                new_S_mc,
                new_E_mc,
                new_A_mc,
                new_I_mc,
                new_R_mc,
                new_S_sc,
                new_E_sc,
                new_A_sc,
                new_I_sc,
                new_ICU_c,
                new_R_sc,
                new_S_sr,
                new_E_sr,
                new_A_sr,
                new_I_sr,
                new_ICU_r,
                new_R_sr,
                new_D])
        except Exception as e:
            print('2')


    @staticmethod
    def get_y_order():
        return ['S_mr',
                'E_mr',
                'A_mr',
                'I_mr',
                'R_mr',
                'S_mc',
                'E_mc',
                'A_mc',
                'I_mc',
                'R_mc',
                'S_sc',
                'E_sc',
                'A_sc',
                'I_sc',
                'ICU_c',
                'R_sc',
                'S_sr',
                'E_sr',
                'A_sr',
                'I_sr',
                'ICU_r',
                'R_sr',
                'D']

    def ODE_function(self, delta_r, delta_c):
        def lda_tot(y, beta):
            try:
                return beta * (effective_A(y, delta_r, delta_c) + effective_I(y, delta_r, delta_c))
            except Exception as e:
                print('3')
            
        def lda_m(y, beta):
            try:
                return (1 - delta_r) * lda_tot(y, beta)
            except Exception as e:
                print('4')

        def lda_s(y, beta):
            try:
                return (1 - delta_c) * lda_tot(y, beta)
            except Exception as e:
                print('5')            

        # For S variables
        def f_S_mr(y, beta):
            try:
                return -lda_m(y, beta) * y[SeairOde.get_y_order().index("S_mr")]
            except Exception as e:
                print('6')            

        def f_S_mc(y, beta):
            try:
                return -lda_s(y, beta) * y[SeairOde.get_y_order().index("S_mc")]
            except Exception as e:
                print('7')            

        def f_S_sc(y, beta):
            try:
                return -lda_s(y, beta) * y[SeairOde.get_y_order().index("S_sc")]
            except Exception as e:
                print('8')            

        def f_S_sr(y, beta):
            try:
                return -lda_m(y, beta) * y[SeairOde.get_y_order().index("S_sr")]
            except Exception as e:
                print('9')            

        # For E variables
        def f_E_mr(y, beta, eps):
            try:
                return lda_m(y, beta) * y[SeairOde.get_y_order().index("S_mr")] - eps * y[SeairOde.get_y_order().index("E_mr")]
            except Exception as e:
                print('10')            

        def f_E_mc(y, beta, eps):
            try:
                return lda_s(y, beta) * y[SeairOde.get_y_order().index("S_mc")] - eps * y[SeairOde.get_y_order().index("E_mc")]
            except Exception as e:
                print('11')            

        def f_E_sc(y, beta, eps):
            try:
                return lda_s(y, beta) * y[SeairOde.get_y_order().index("S_sc")] - eps * y[SeairOde.get_y_order().index("E_sc")]
            except Exception as e:
                print('12')            

        def f_E_sr(y, beta, eps):
            try:
                return lda_m(y, beta) * y[SeairOde.get_y_order().index("S_sr")] - eps * y[SeairOde.get_y_order().index("E_sr")]
            except Exception as e:
                print('13')            

        # For A variables
        def f_A_mr(y, eps, sigma):
            try:
                return eps * y[SeairOde.get_y_order().index("E_mr")] - sigma * y[SeairOde.get_y_order().index("A_mr")]
            except Exception as e:
                print('14')            

        def f_A_mc(y, eps, sigma):
            try:
                return eps * y[SeairOde.get_y_order().index("E_mc")] - sigma * y[SeairOde.get_y_order().index("A_mc")]
            except Exception as e:
                print('16')            

        def f_A_sc(y, eps, sigma):
            try:
                return eps * y[SeairOde.get_y_order().index("E_sc")] - sigma * y[SeairOde.get_y_order().index("A_sc")]
            except Exception as e:
                print('17')            

        def f_A_sr(y, eps, sigma):
            try:
                return eps * y[SeairOde.get_y_order().index("E_sr")] - sigma * y[SeairOde.get_y_order().index("A_sr")]
            except Exception as e:
                print('18')            

        # For I variables
        def f_I_mr(y, sigma, gamma_m):
            try:
                return sigma * y[SeairOde.get_y_order().index("A_mr")] - gamma_m * y[SeairOde.get_y_order().index("I_mr")]
            except Exception as e:
                print('19')            

        def f_I_mc(y, sigma, gamma_m):
            try:
                return sigma * y[SeairOde.get_y_order().index("A_mc")] - gamma_m * y[SeairOde.get_y_order().index("I_mc")]
            except Exception as e:
                print('20')            

        def f_I_sc(y, sigma, eta):
            try:
                return sigma * y[SeairOde.get_y_order().index("A_sc")] - eta * y[SeairOde.get_y_order().index("I_sc")]
            except Exception as e:
                print('21')            

        def f_I_sr(y, sigma, eta):
            try:
                return sigma * y[SeairOde.get_y_order().index("A_sr")] - eta * y[SeairOde.get_y_order().index("I_sr")]
            except Exception as e:
                print('22')            

        # For ICU variables
        def f_ICU_c(y, eta, gamma_s, alpha):
            try:
                return eta * y[SeairOde.get_y_order().index("I_sc")] - (gamma_s + alpha) * y[SeairOde.get_y_order().index("ICU_c")]
            except Exception as e:
                print('23')            

        def f_ICU_r(y, eta, gamma_s, alpha):
            try:
                return eta * y[SeairOde.get_y_order().index("I_sr")] - (gamma_s + alpha) * y[SeairOde.get_y_order().index("ICU_r")]
            except Exception as e:
                print('24')            

        # For R variables
        def f_R_mr(y, gamma_m):
            try:
                return gamma_m * y[SeairOde.get_y_order().index("I_mr")]
            except Exception as e:
                print('26')            

        def f_R_mc(y, gamma_m):
            try:
                return gamma_m * y[SeairOde.get_y_order().index("I_mc")]
            except Exception as e:
                print('28')            

        def f_R_sc(y, gamma_s):
            try:
                return gamma_s * y[SeairOde.get_y_order().index("ICU_c")]
            except Exception as e:
                print('29')            

        def f_R_sr(y, gamma_s):
            try:
                return gamma_s * y[SeairOde.get_y_order().index("ICU_r")]
            except Exception as e:
                print('30')            

        # For the D variable
        def f_D(y, alpha):
            try:
                return alpha*ICU(y)
            except Exception as e:
                print('31')            

        # Full function
        return lambda t, y: [f_S_mr(y, self.parameters['beta']),
                             f_E_mr(y, self.parameters['beta'], self.parameters['eps']),
                             f_A_mr(y, self.parameters['eps'], self.parameters['sigma']),
                             f_I_mr(y, self.parameters['sigma'], self.parameters['gamma_m']),
                             f_R_mr(y, self.parameters['gamma_m']),
                             f_S_mc(y, self.parameters['beta']),
                             f_E_mc(y, self.parameters['beta'], self.parameters['eps']),
                             f_A_mc(y, self.parameters['eps'], self.parameters['sigma']),
                             f_I_mc(y, self.parameters['sigma'], self.parameters['gamma_m']),
                             f_R_mc(y, self.parameters['gamma_m']),
                             f_S_sc(y, self.parameters['beta']),
                             f_E_sc(y, self.parameters['beta'], self.parameters['eps']),
                             f_A_sc(y, self.parameters['eps'], self.parameters['sigma']),
                             f_I_sc(y, self.parameters['sigma'], self.parameters['eta']),
                             f_ICU_c(y, self.parameters['eta'], self.parameters['gamma_s'], self.parameters['alpha']),
                             f_R_sc(y, self.parameters['gamma_s']),
                             f_S_sr(y, self.parameters['beta']),
                             f_E_sr(y, self.parameters['beta'], self.parameters['eps']),
                             f_A_sr(y, self.parameters['eps'], self.parameters['sigma']),
                             f_I_sr(y, self.parameters['sigma'], self.parameters['eta']),
                             f_ICU_r(y, self.parameters['eta'], self.parameters['gamma_s'], self.parameters['alpha']),
                             f_R_sr(y, self.parameters['gamma_s']),
                             f_D(y, self.parameters['alpha'])]

    def prop_ODE(self, delta_r, delta_c, y_init, final_time, ndiscr):
        try:
            t = np.linspace(0, final_time, ndiscr)
            sol = odeint(self.ODE_function(delta_r, delta_c), y_init, t, tfirst=True)
            return sol
        except Exception as e:
            print('32')


    def get_N_0(self):
        return self.initial_conditions['N_0']


######################
## S
######################
def S_m(y):
    try:
        return y[SeairOde.get_y_order().index("S_mr")]+y[SeairOde.get_y_order().index("S_mc")]
    except Exception as e:
        print('33')


def S_s(y):
    try:
        return y[SeairOde.get_y_order().index("S_sc")]+y[SeairOde.get_y_order().index("S_sr")]
    except Exception as e:
        print('34')    


def S(y):
    try:
        return S_m(y) + S_s(y)
    except Exception as e:
        print('36')    


######################
## E
######################
def E_m(y):
    try:
        return y[SeairOde.get_y_order().index("E_mr")]+y[SeairOde.get_y_order().index("E_mc")]
    except Exception as e:
        print('37')    


def E_s(y):
    try:
        return y[SeairOde.get_y_order().index("E_sc")]+y[SeairOde.get_y_order().index("E_sr")]
    except Exception as e:
        print('38')    


def E(y):
    try:
        return E_m(y) + E_s(y)
    except Exception as e:
        print('39')    


######################
## A
######################
def A_m(y):
    try:
        return y[SeairOde.get_y_order().index("A_mr")]+y[SeairOde.get_y_order().index("A_mc")]
    except Exception as e:
        print('40')    


def A_s(y):
    try:
        return y[SeairOde.get_y_order().index("A_sc")]+y[SeairOde.get_y_order().index("A_sr")]
    except Exception as e:
        print('41')    


def A_cm(y):
    try:
        return y[SeairOde.get_y_order().index("A_mr")]+y[SeairOde.get_y_order().index("A_sr")]
    except Exception as e:
        print('42')    


def A_cs(y):
    try:
        return y[SeairOde.get_y_order().index("A_mc")]+y[SeairOde.get_y_order().index("A_sc")]
    except Exception as e:
        print('43')    


def A(y):
    try:
        return A_m(y) + A_s(y)
    except Exception as e:
        print('44')    


######################
## I
######################
def I_m(y):
    try:
        return y[SeairOde.get_y_order().index("I_mr")]+y[SeairOde.get_y_order().index("I_mc")]
    except Exception as e:
        print('45')    


def I_s(y):
    try:
        return y[SeairOde.get_y_order().index("I_sc")]+y[SeairOde.get_y_order().index("I_sr")]
    except Exception as e:
        print('46')    


def I_cm(y):
    try:
        return y[SeairOde.get_y_order().index("I_mr")]+y[SeairOde.get_y_order().index("I_sr")]
    except Exception as e:
        print('47')    


def I_cs(y):
    try:
        return y[SeairOde.get_y_order().index("I_mc")]+y[SeairOde.get_y_order().index("I_sc")]
    except Exception as e:
        print('48')    


def I(y):
    try:
        return I_m(y) + I_s(y)
    except Exception as e:
        print('49')    


######################
## ICU
######################
def ICU(y):
    try:
        return y[SeairOde.get_y_order().index("ICU_c")] + y[SeairOde.get_y_order().index("ICU_r")]
    except Exception as e:
        print('50')    


######################
## R
######################
def R_m(y):
    try:
        return y[SeairOde.get_y_order().index("R_mr")]+y[SeairOde.get_y_order().index("R_mc")]
    except Exception as e:
        print('51')    


def R_s(y):
    try:
        return y[SeairOde.get_y_order().index("R_sc")]+y[SeairOde.get_y_order().index("R_sr")]
    except Exception as e:
        print('52')    


def R(y):
    try:
        return R_m(y) + R_s(y)
    except Exception as e:
        print('53')    


######################
## N
######################
def N(y):
    try:
        return S(y)+E(y)+A(y)+I(y)+ICU(y)+R(y)
    except Exception as e:
        print('54')    


######################
## Effective contact
######################
def effective_A(y,delta_r,delta_c):
    try:
        return (1-delta_r)*A_cm(y) + (1-delta_c)*A_cs(y)
    except Exception as e:
        print('55')    


def effective_I(y,delta_r,delta_c):
    try:
        return (1-delta_r)*I_cm(y) + (1-delta_c)*I_cs(y)
    except Exception as e:
        print('56')   

In [ ]:
#from sklearn import metrics
from os import path

In [ ]:
def plot_densities(alpha_p, beta_p, alpha_n, beta_n, fig_path=None):
    try:
        prob_grid = np.linspace(0,1,10000)
        positive_pdf = stat.beta.pdf(prob_grid, alpha_p, beta_p)
        negative_pdf = stat.beta.pdf(prob_grid, alpha_n, beta_n)
        plt.plot(prob_grid, positive_pdf, 'b', label = 'Severe symptoms')
        plt.plot(prob_grid, negative_pdf, 'r', label = 'Mild symptoms')
        plt.xlabel('a')
        plt.title('Probability densities for severe and mild symptoms')
        plt.legend(loc='best')
        if fig_path is None:
            plt.show()
        else:
            plt.savefig(path.join(fig_path, 'populations_densities.png'))
    except Exception as e:
        print('57') 

In [ ]:
#def compute_auc(alpha_p, beta_p, alpha_n, beta_n):
#    try:
#        a = np.linspace(0,1,1000)
#        q_FP = []
#        q_TP = []
#        for i in range(1000):
#            q_FP.append(1-stat.beta.cdf(a[i], alpha_n, beta_n))
#            q_TP.append(1-stat.beta.cdf(a[i], alpha_p, beta_p))
#        return metrics.auc(q_FP, q_TP)
#    except Exception as e:
#        print('58')

In [ ]:
def fct_overwhelm(z, I_max, number_points_per_day):
    try:
        fct_I_max = I_max*np.ones(z.size)
        integral = (1/number_points_per_day)*sum((z-fct_I_max)*(z-fct_I_max>0))
        return (1/30)*(1/I_max)*integral
    except Exception as e:
        print('59')

def overwhelm_vs_release(I_max, cdf_p, cdf_n, T, delta_r, delta_c, grid_prop, p=0.995, N_0=67*10**6, R_0=4*10**6, Inf_0 = 20*10**4, ICU_0=4*10**3, number_points_per_day = 500):
    try:
        overwhelm = []
        for grid_prop_i in grid_prop:
            q = compute_q(p, cdf_p, cdf_n, np.array([grid_prop_i]))
            seair = SeairOde(p=p, N_0=N_0, R_0=R_0, Inf_0=Inf_0, ICU_0=ICU_0, number_points_per_day=number_points_per_day) # TODO : study **kwargs
            [t, sol] = seair.progressive_release(T, q, delta_r, delta_c)
            overwhelm_temp = fct_overwhelm(sol[:, SeairOde.get_y_order().index("ICU_c")] +
                                           sol[:, SeairOde.get_y_order().index("ICU_r")],
                                           I_max,
                                           number_points_per_day)
            overwhelm.append(overwhelm_temp)
        return overwhelm
    except Exception as e:
        print('60')


def generate_time_prop_fixed_frequency(n_days, release_rate):
    try:
        n = int(np.ceil(100 / release_rate))
        T = np.arange(n_days, n_days*(n+1), n_days)
        prop = np.linspace(release_rate / 100, n * release_rate / 100, n)
        prop[-1] = 1.0
        return T, prop
    except Exception as e:
        print('61')

def dichotomy(f):
    try:
        g, d = 0., 1.
        eps = 10**(-5)
        while d-g > eps:
            m = (g+d)/2
            if f(m) == 0:
                return m
            elif f(g) * f(m) < 0:
                d=m
            else:
                g=m
        return (g+d)/2
    except Exception as e:
        print('62')



def invert(p, cdf_p, cdf_n, prop):
    try:
        a = []
        for prop_i in prop:
            fct_temp = lambda t: p*cdf_n(t)+(1-p)*cdf_p(t)-prop_i
            a.append(dichotomy(fct_temp))
        return np.array(a)
    except Exception as e:
        print('63')

def compute_q(p, cdf_p, cdf_n, prop):
    try:
        a = invert(p, cdf_p, cdf_n, prop)
        q = []
        for a_i in a:
            # q_FP : people non released but who will have mild symptoms
            q_FP = 1-cdf_n(a_i)
            # q_FN : people released but who will have severe symptoms
            q_FN = cdf_p(a_i)
            q.append(np.array([q_FP, q_FN]))
        return q
    except Exception as e:
        print('64') 

In [ ]:
#added 20Jul2020
def exists(path): 
    try:
        r = requests.head(path)
        return r.status_code == requests.codes.ok
    except Exception as e:
        print('65')        

In [ ]:
def get_world_population(): 
    try:
        #https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv
        global df_world_population

        year=datetime.datetime.today().year
        #url="https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv"    
        df_local1=pd.read_csv('WPP2019_TotalPopulationBySex.csv')
        df_local1=df_local1[df_local1['Time']==year]    
        df_local2=df_local1.filter(['Location', 'PopTotal'], axis=1)
        df_local2['PopTotal']=df_local2['PopTotal']*1000
        df_world_population=df_local2.groupby('Location')['PopTotal'].mean()
        df_world_population=df_world_population.to_frame().reset_index()
    except Exception as e:
        print('66')  

In [ ]:
def get_covid_country():
    try:
        #this is for the countries dropdown menu
        global df_selection_country

        url1="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/08-15-2020.csv"
        df1=pd.read_csv(url1)   
        df1 = df1.drop(df1[(df1.Country_Region == 'Diamond Princess') |
                           (df1.Country_Region == 'South Sudan') |
                           (df1.Country_Region == 'Congo (Brazzaville)') |
                           (df1.Country_Region == 'Congo (Kinshasa)') |
                           (df1.Country_Region == "Cote d'Ivoire") |
                           (df1.Country_Region == 'Holy See') |
                           (df1.Country_Region == 'Kosovo') |
                           (df1.Country_Region == 'Liechtenstein') |
                           (df1.Country_Region == 'MS Zaandam') |
                           (df1.Country_Region == 'West Bank and Gaza')
                          ].index)        
        df2=[x for x in df1['Country_Region'].unique() if x!='Canada']    
        #df2=[x for x in df1['Country_Region'].unique() if x!='US']    
        #df2.insert(0, 'Global')
        df2.insert(0, 'Select Country')
        df2.insert(1, 'Canada')
        #df2.insert(0, 'US')                
        df_selection_country=pd.DataFrame(df2, columns=['Country_Region'])        
    except Exception as e:
        print(e)
        print('67')       

In [ ]:
def get_covid_dates():
    try:
        global df_date

        date = datetime.date(2020,3,22) #this is the oldest date where Canada was first recorded
        date_lst = []

        date_lst.append('Select Date')

        for date_counter in range(1, (date.today() - date).days, 1):
            today = date.today() - timedelta(date_counter)  
            today = today.strftime("%m-%d-%Y")
            date_lst.append(today)    

        df_date = pd.DataFrame(date_lst, columns=["Dates"]) 
    except Exception as e:
        print('68')   

In [ ]:
def show_stats(name, date, recovered, infected, icu, deaths, inc_rate, case_fatal_ratio, country_code):
    try:
        recovered_format=format(recovered.astype(int), ',d')
        infected_format=format(infected.astype(int), ',d')
        icu_format=format(icu.astype(int), ',d')
        deaths_format=format(deaths.astype(int), ',d')
        
        
        
        if name=='Namibia':
            country_flag='<img src="https://www.countryflags.io/NA/flat/64.png" border="1"/>'
        else:
            country_flag='<img src="https://www.countryflags.io/'+str(country_code)+'/flat/64.png" border="1"/>'
        
        popup='''<table width='100%' cellspacing='0' cellpadding='0'>
                    <tr>
                        <td colspan='2' align='center'>'''+country_flag+'''</td>
                    </tr>
                    <tr>
                        <td><h4><b>'''+name+'''</b></h4></td>
                        <td align='right'>'''+date+'''</td>
                    </tr>
                </table>
                        <ul>                        
                            <li>Recovered: '''+recovered_format+'''</li>
                            <li>Infected: '''+infected_format+'''</li>
                            <li>ICU: '''+icu_format+'''</li>
                            <li>Deaths: '''+deaths_format+'''</li>
                            
                        </ul>
                        '''
                            #<li>Incidence Rate: '''+str(round(inc_rate, 4))+'''</li>
                            #<li>Case-Fatality Ratio: '''+str(round(case_fatal_ratio, 4))+'''</li>
        return popup
    except Exception as e:
        print('69')
        #popup=name + " | " + date + " | " + recovered + " | " + infected + " | " + icu + " | " + deaths + " | " + inc_rate + " | " + case_fatal_ratio + " | " + country_code
        #pass

In [ ]:
def get_country_hospital_beds():
    try:
        global country_hospital_beds    
        country_hospital_beds = pd.read_csv("Hospital_Beds_Per_1000.csv", encoding='utf-8') 
    except Exception as e:
        print('70')

In [ ]:
def cdf(data):    
    def inner_cdf(t):
        try:
            return np.sum(data<=t)/data.shape[0]
        except Exception as e:
            print('71')        
    return inner_cdf

In [ ]:
def cdf_beta_n(alpha_n, beta_n):
    def inner_cdf_beta_n(t):
        try:
            return stat.beta.cdf(t, alpha_n, beta_n)
        except Exception as e:
            print('72')        
    return inner_cdf_beta_n

In [ ]:
def cdf_beta_p(alpha_p, beta_p):
    def inner_cdf_beta_p(t):
        try:
            return stat.beta.cdf(t, alpha_p, beta_p)
        except Exception as e:
            print('73')        
    return inner_cdf_beta_p

In [ ]:
def get_population_country_name(country):
    try:
        if country=="Bolivia":
            return "Bolivia (Plurinational State of)"
        elif country=="Brunei":
            return "Brunei Darussalam"
        elif country=="Burma":
            return "Myanmar"
        elif country=="Cote d'Ivoire":
            return "CÃ´te d'Ivoire"
        elif country=="Iran":
            return "Iran (Islamic Republic of)"
        elif country=="Korea, South":
            return "Republic of Korea"
        elif country=="Laos":
            return "Lao People's Democratic Republic"
        elif country=="Moldova":
            return "Republic of Moldova"
        elif country=="Russia":
            return "Russian Federation"
        elif country=="Syria":
            return "Syrian Arab Republic"
        elif country=="Taiwan*":
            return "China, Taiwan Province of China"
        elif country=="Tanzania":
            return "United Republic of Tanzania"
        elif country=="US":
            return "United States of America"
        elif country=="Venezuela":
            return "Venezuela (Bolivarian Republic of)"
        elif country=="Vietnam":
            return "Viet Nam"
        else:
            return country  
    except Exception as e:
        print('74')        

In [ ]:
def num_of_death(N_0, R_0, Inf_0, ICU_0, p, I_max, T_0, prop_0, d_r, d_c): 
    try:
        #FB start: convert the list of string to numeric then to np.array
        prop_0 = prop_0.split(',')    
        for i in range(0, len(prop_0)):
            prop_0[i] = float(prop_0[i])
        prop = np.array(prop_0)    
        load_progress.value=load_progress.value+1

        T_0 = T_0.split(',')    
        for i in range(0, len(T_0)):
            T_0[i] = int(T_0[i])
        T = np.array(T_0)   
        load_progress.value=load_progress.value+1
        #FB end: convert the list of string to numeric then to np.array

        q = compute_q(p, cdf(severe), cdf(mild), prop)
        seair = SeairOde(p = p,
                     N_0 = N_0,
                     R_0= R_0,
                     Inf_0 = Inf_0,
                     ICU_0 = ICU_0) 
        load_progress.value=load_progress.value+1

        #[t, sol] = seair.progressive_release(T, q, delta_r, delta_c)   
        [t, sol] = seair.progressive_release(T, q, d_r, d_c)   
        load_progress.value=load_progress.value+1

        g1 = widgets.Output()  
        with g1:
            plt.plot(t, sol[:,SeairOde.get_y_order().index("D")], 'b')
            plt.xlabel('t')
            plt.ylabel('Number of deaths')
            plt.title('\nSEAIR response of number of deaths')
            plt.grid()    
            plt.show()        

        g2 = widgets.Output()  
        with g2:
            plt.stackplot(t, 
                  np.vstack([sol[:,SeairOde.get_y_order().index("ICU_c")], 
                             sol[:,SeairOde.get_y_order().index("ICU_r")]]), 
                  labels=['High Risk', 'Low Risk'])
            plt.title('Saturation of ICU\n(Stacked)')
            plt.ylabel('Number of people in ICU')
            plt.plot(t, I_max*np.ones(t.size), 'brown', label='ICU Saturation')
            #plt.legend(loc='upper left')
            plt.legend(loc='center right')
            plt.xlabel('t')
            plt.grid()
            plt.show()               

        g3 = widgets.Output()  
        with g3:
            plt.stackplot(t, 
                  np.vstack([sol[:,SeairOde.get_y_order().index("R_sr")]/seair.get_N_0(),
                             sol[:,SeairOde.get_y_order().index("R_sc")]/seair.get_N_0()]), 
                  labels=['Low Risk', 'High Risk'])
            plt.ylabel('Stacked Ratio of total population')
            #plt.legend(loc='upper left')
            plt.legend(loc='lower right')
            plt.title('SEAIR response for recovered people\n(People infected by the disease, survived and are now immune)\nSevere Infections')
            plt.xlabel('t')
            plt.grid()
            plt.show()    

        g4 = widgets.Output() 
        with g4:
            plt.stackplot(t, 
                  np.vstack([sol[:,SeairOde.get_y_order().index("R_mr")]/seair.get_N_0(), 
                             sol[:,SeairOde.get_y_order().index("R_mc")]/seair.get_N_0()]), 
                  labels=['Low Risk', 'High Risk'])
            plt.ylabel('Stacked Ratio of total population')
            #plt.legend(loc='upper left')
            plt.legend(loc='lower right')
            plt.title('SEAIR response for recovered people\n(People infected by the disease, survived and are now immune)\nMild Infections')
            plt.xlabel('t')
            plt.grid()
            plt.show()   

        g5 = widgets.Output() 
        with g5:
            plt.stackplot(t, 
                  np.vstack([sol[:,SeairOde.get_y_order().index("I_sc")], 
                             sol[:,SeairOde.get_y_order().index("I_sr")]]), 
                  labels=['High Risk', 'Low Risk'])
            plt.xlabel('t')
            plt.ylabel('Number severe infections')
            #plt.legend(loc='upper left')
            plt.legend(loc='center right')
            plt.title('SEAIR response for severe infections\n(Stacked)')
            plt.grid()
            plt.show()        

        lbl_legends=widgets.Output()
        with lbl_legends:
            print("Legend:")
            print('delta_r = how much released people have lowered their exits from home.')
            print('delta_c = how much people in lockdown have lowered their exits from home.')
            print('Capacity = approximate total hospital bed count.')
            print('Proportion = array of cumulated ratio of released population along time defined in T. The first element refers to the population relased at time 0.')
            print('Ratio, mild = proportion of people with mild symptoms.')
            print('T_0 = array representing times of releases, the last time in the array is the chosen horizon time.')
            print('e.g.: if T=[100, 300] and proportion=[0.2, 0.8], 20% of the population is allowed to go out at time 0, and then 60% more at time 101. 300 being the final horizon time.')

        display(HBox([g1, g2, g5]))        
        display(HBox([g3, g4]))
        display(lbl_legends)        
        load_progress.value=load_progress.value+1
    except Exception as e:
        lbl_T_0=widgets.Label(value='Please supply 3 positive integer values for T_0 separated by comma. e.g.: 50, 120, 200')        
        lbl_prop=widgets.Label(value='Please supply 3 positive float values for Proportion separated by comma. e.g.: 0.3, 0.5, 0.7')        
        display(VBox([lbl_T_0, lbl_prop]))        
    
def interact_num_of_death(N_0_local, R_0_local, Inf_0_local, ICU_0_local, num_beds):
    p = widgets.FloatSlider(description='Ratio, mild', min=0, max=1, step=0.05, value=0.995, continuous_update=False)
    T_0 = widgets.Text(value='50, 120, 200', placeholder='Supply T_0', description='T_0', continuous_update=False)
    prop_0 = widgets.Text(value='0.3, 0.7, 1.0', placeholder='Supply proportion', description='Proportion', continuous_update=False)
    N_0 = widgets.IntSlider(description='Population', min=N_0_local//2, max=N_0_local*2, step=1, value=N_0_local, continuous_update=False)
    R_0 = widgets.IntSlider(description='Recovered', min=0, max=R_0_local*2, step=1, value=R_0_local, continuous_update=False)
    Inf_0 = widgets.IntSlider(description='Infected', min=0, max=Inf_0_local*2, step=1, value=Inf_0_local, continuous_update=False)
    ICU_0 = widgets.IntSlider(description='ICU', min=0, max=ICU_0_local*2, step=1, value=ICU_0_local, continuous_update=False)
    d_r = widgets.FloatSlider(description='delta_r', min=0, max=1, step=0.05, value=0.1, continuous_update=False)
    d_c = widgets.FloatSlider(description='delta_c', min=0, max=1, step=0.05, value=0.6, continuous_update=False)
    
    if chosen_country=='Canada' or chosen_country=='US':
        I_max = widgets.IntSlider(description='Capacity', min=1, max=num_beds*10, step=1, value=num_beds, continuous_update=False)
        #g1 = interactive(num_of_death,
        #                 p = widgets.FloatSlider(description='Ratio, mild', min=0, max=1, step=0.05, value=0.995, continuous_update=False),         
        #                 T_0 = widgets.Text(value='50, 120, 200', placeholder='Supply T_0', description='T_0:', continuous_update=False),
        #                 prop_0 = widgets.Text(value='0.3, 0.7, 1.0', placeholder='Supply proportion', description='Proportion', continuous_update=False),
        #                 N_0 = widgets.IntSlider(description='Population', min=N_0_local//2, max=N_0_local*2, step=1, value=N_0_local, continuous_update=False),
        #                 R_0 = widgets.IntSlider(description='Recovered', min=0, max=R_0_local*2, step=1, value=R_0_local, continuous_update=False),
        #                 Inf_0 = widgets.IntSlider(description='Infected', min=0, max=Inf_0_local*2, step=1, value=Inf_0_local, continuous_update=False),
        #                 ICU_0 = widgets.IntSlider(description='ICU', min=0, max=ICU_0_local*2, step=1, value=ICU_0_local, continuous_update=False),
        #                 #delta_r = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1),
        #                 #delta_c = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.6),
        #                 I_max = widgets.IntSlider(description='Capacity', min=1, max=num_beds*10, step=1, value=num_beds, continuous_update=False)
        #                )    
    else:
        I_max = widgets.IntSlider(description='Capacity', min=1, max=((N_0_local/10000) * num_beds * 10).astype(int), step=1, value=((N_0_local/10000) * num_beds).astype(int), continuous_update=False)                
        #g1 = interactive(num_of_death,
                         #p = widgets.FloatSlider(description='Ratio, mild', min=0, max=1, step=0.05, value=0.995, continuous_update=False),         
                         #T_0 = widgets.Text(value='50, 120, 200', placeholder='Supply T_0', description='T_0:', continuous_update=False),
                         #prop_0 = widgets.Text(value='0.3, 0.7, 1.0', placeholder='Supply proportion', description='Proportion', continuous_update=False),
                         #N_0 = widgets.IntSlider(description='Population', min=N_0_local//2, max=N_0_local*2, step=1, value=N_0_local, continuous_update=False),
                         #R_0 = widgets.IntSlider(description='Recovered', min=0, max=R_0_local*2, step=1, value=R_0_local, continuous_update=False),
                         #Inf_0 = widgets.IntSlider(description='Infected', min=0, max=Inf_0_local*2, step=1, value=Inf_0_local, continuous_update=False),
                         #ICU_0 = widgets.IntSlider(description='ICU', min=0, max=ICU_0_local*2, step=1, value=ICU_0_local, continuous_update=False),
                         ##delta_r = widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1),
                         ##delta_c = widgets.FloatSlider(min=0,max=1,step=0.1,value=0.6),
                         #I_max = widgets.IntSlider(description='Capacity', min=1, max=((N_0_local/10000) * num_beds * 10).astype(int), step=1, value=((N_0_local/10000) * num_beds).astype(int), continuous_update=False)
        #                )            
        
    ui1=widgets.HBox([N_0, R_0, Inf_0, ICU_0])        
    ui2=widgets.HBox([p, I_max, d_r, d_c])
    ui3=widgets.HBox([T_0, prop_0])
    out=widgets.interactive_output(num_of_death, {'N_0': N_0, 'R_0': R_0, 'Inf_0': Inf_0, 'ICU_0': ICU_0, 'p': p, 'I_max': I_max, 'T_0': T_0, 'prop_0': prop_0, 'd_r': d_r, 'd_c': d_c})                
    
    display(ui1, ui2, ui3, out)
    #display(g1)
    load_progress.value=load_progress.value+1

In [ ]:
## define plot functions so it can be used by ipywidgets
## the arguements inside the function is what we want to make its interactive
def contact_rate(filler1=' ', filler2=' ', delta_r=0.1,delta_c=0.6):
    try:
        grid_prop = np.linspace(0,1,1000)
        av_r= 1-(1-delta_r)*((1-delta_r)*grid_prop +(1-delta_c)*(1-grid_prop))
        av_c= 1-(1-delta_c)*((1-delta_r)*grid_prop +(1-delta_c)*(1-grid_prop))    
        plt.plot(grid_prop, av_r, 'r', label='Released')
        plt.plot(grid_prop, av_c, 'b', label='Locked Down')
        plt.legend(loc='best')
        plt.title('Contacts : Contact rates in each subpopulation \ngiven the ratio of the population released')
        plt.xlabel('Proportion of people released')
        plt.ylabel('Reduction of contact rate')
        plt.grid()
        plt.show()
    except Exception as e:
        print('75')
        
def plot_densities(alpha_p, beta_p, alpha_n, beta_n):
    try:
        prob_grid = np.linspace(0,1,10000)
        positive_pdf = stat.beta.pdf(prob_grid, alpha_p, beta_p)
        negative_pdf = stat.beta.pdf(prob_grid, alpha_n, beta_n)
        plt.plot(prob_grid, positive_pdf, 'b', label = 'Severe symptoms')
        plt.plot(prob_grid, negative_pdf, 'r', label = 'Mild symptoms')
        plt.xlabel('a')
        plt.title('\nProbability densities for severe and mild symptoms')
        plt.legend(loc='best')
        plt.grid()
        plt.show()
    except Exception as e:
        print('76')    
        
def mild_severe_symptom_score(filler1=' ', filler2=' ', filler3=' ', bins=50):
    try:
        mild = np.genfromtxt('data/mild_a.csv', delimiter=',')
        severe = np.genfromtxt('data/severe_a.csv', delimiter=',')
        plt.hist(mild, bins=bins, density=True, facecolor='red', alpha=0.5, label="Mild symptoms")
        plt.hist(severe, bins=bins, density=True, facecolor='blue', alpha=0.5, label="Severe symptoms")
        plt.legend(loc='best')
        plt.xlabel('a')
        plt.ylabel('Cases in population')
        plt.title('\nDensities for severe and mild symptoms')
        plt.grid()
        plt.show()
    except Exception as e:
        print('77')
    
def call_general_graphs():    
    g1 = interactive(contact_rate, 
             filler1=widgets.Label(value=' '),        
             filler2=widgets.Label(value=' '),        
             delta_r=widgets.FloatSlider(min=0, max=1, step=0.1, value=0.1, continuous_update=False), 
             delta_c=widgets.FloatSlider(min=0,max=1,step=0.1,value=0.6, continuous_update=False)) 

    g2 = interactive(plot_densities,
         alpha_p = widgets.FloatSlider(min=0, max=5, step=0.1, value=4.5, continuous_update=False),
         beta_p = widgets.FloatSlider(min=0, max=5, step=0.1, value=2, continuous_update=False), 
         alpha_n = widgets.FloatSlider(min=0, max=5, step=0.1, value=2, continuous_update=False), 
         beta_n = widgets.FloatSlider(min=0, max=5, step=0.1, value=4.5, continuous_update=False))    

    g3 = interactive(mild_severe_symptom_score,
         filler1=widgets.Label(value=' '),        
         filler2=widgets.Label(value=' '),        
         filler3=widgets.Label(value=' '),        
         bins = widgets.IntSlider(min=1, max=100, step=1, value=50, continuous_update=False))  
    
    display(HBox([g1, g2, g3]))

In [ ]:
def display_graphs(df_covid_local):
    try:
        global output
        global tabs


        confirmed_total = 0
        recovered_total = 0
        active_total = 0
        N = 0
        total_beds = 0    

        df_selected_country = df_covid_local[df_covid_local['Country_Region']==chosen_country]
        load_progress.value=load_progress.value+1
        output = widgets.Output()  

        if chosen_country!='Canada':
            country_hospital_beds_local=country_hospital_beds[country_hospital_beds['Country_Name']==chosen_country]                    
            load_progress.value=load_progress.value+1

            out1 = widgets.Output()
            tabs = widgets.Tab(children = [out1])
            tabs.set_title(0, chosen_country)

            with output:    
                with out1:
                    df1 = df_selected_country
                    load_progress.value=load_progress.value+1

                    if country_hospital_beds_local.empty==False:
                        prov_beds = (country_hospital_beds_local['Beds'].sum()*10).astype(int)
                    else:
                        prov_beds=1
                    load_progress.value=load_progress.value+1

                    if df1.empty==False:
                        confirmed = df1["Confirmed"].sum()
                        load_progress.value=load_progress.value+1
                        recovered = df1["Recovered"].sum()
                        load_progress.value=load_progress.value+1
                        active = df1["Active"].sum()
                        load_progress.value=load_progress.value+1
                        df_pop = df_world_population[df_world_population['Location']==get_population_country_name(chosen_country)]
                        load_progress.value=load_progress.value+1
                        N = df_pop['PopTotal'].iloc[0]

                        interact_num_of_death(N.astype(np.int64), recovered, confirmed, active, prov_beds)   
        else:
            out1 = widgets.Output()
            out2 = widgets.Output()
            out3 = widgets.Output()
            out4 = widgets.Output()
            out5 = widgets.Output()
            out6 = widgets.Output()
            out7 = widgets.Output()
            out8 = widgets.Output()
            out9 = widgets.Output()
            out10 = widgets.Output()
            out11 = widgets.Output()
            out12 = widgets.Output()
            out13 = widgets.Output()

            tabs = widgets.Tab(children = [out1, out2, out3, out4, out5, out6, out7, out8, out9, out10, out11, out12, out13])

            tabs.set_title(0, 'Canada')
            tabs.set_title(1, 'AB')
            tabs.set_title(2, 'BC')
            tabs.set_title(3, 'MB')
            tabs.set_title(4, 'NB')
            tabs.set_title(5, 'NL')
            tabs.set_title(6, 'NS')
            tabs.set_title(7, 'NW')
            tabs.set_title(8, 'ON')
            tabs.set_title(9, 'PE')
            tabs.set_title(10, 'QC')
            tabs.set_title(11, 'SK')
            tabs.set_title(12, 'YK')

            #Canadian provinces' population taken from https://worldpopulationreview.com/canadian-provinces
            confirmed_total = 0
            recovered_total = 0
            active_total = 0
            N = 0
            total_beds = 0

            #Canadian provinces' hospital capacities data: https://www.cihi.ca/sites/default/files/document/beds-staffed-and-in-operation-2018-2019-en-web.xlsx
            can_prov_hosp_bed = pd.read_csv("Canada_Province_Hospital_Beds.csv", sep=",", encoding="cp1252")
            load_progress.value=load_progress.value+1
            can_prov_hosp_bed['Total'] = pd.to_numeric(can_prov_hosp_bed['Total'])
            load_progress.value=load_progress.value+1
            #Per country: https://www.who.int/data/gho/data/indicators/indicator-details/GHO/hospital-beds-(per-10-000-population)

            with output:        
                with out2: #AB                      
                    df1 = df_selected_country[df_selected_country['Province_State']=='Alberta']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='AB']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active

                        N = 4345737 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out3: #BC
                    df1 = df_selected_country[df_selected_country['Province_State']=='British Columbia']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='BC']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 5020302 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out4: #MB        
                    df1 = df_selected_country[df_selected_country['Province_State']=='Manitoba']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='MB']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 1360396 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out5: #NB
                    df1 = df_selected_country[df_selected_country['Province_State']=='New Brunswick']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='NB']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 772094 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out6: #NL
                    df1 = df_selected_country[df_selected_country['Province_State']=='Newfoundland and Labrador']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='NL']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 523790 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out7: #NS
                    df1 = df_selected_country[df_selected_country['Province_State']=='Nova Scotia']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='NS']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 965382 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out8: #NW
                    df1 = df_selected_country[df_selected_country['Province_State']=='Northwest Territories']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='NW']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 44598 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out9: #ON
                    df1 = df_selected_country[df_selected_country['Province_State']=='Ontario']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='ON']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 14446515 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out10: #PEI
                    df1 = df_selected_country[df_selected_country['Province_State']=='Prince Edward Island']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='PE']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active                

                        N = 154748 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out11: #QC
                    df1 = df_selected_country[df_selected_country['Province_State']=='Quebec']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='QC']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active

                        N = 8433301 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out12: #SK
                    df1 = df_selected_country[df_selected_country['Province_State']=='Saskatchewan']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='SK']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active

                        N = 1168423 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out13: #YK
                    df1 = df_selected_country[df_selected_country['Province_State']=='Yukon']
                    load_progress.value=load_progress.value+1
                    sub = can_prov_hosp_bed[can_prov_hosp_bed["Province"]=='YK']
                    load_progress.value=load_progress.value+1
                    prov_beds = sub['Total'].sum()
                    load_progress.value=load_progress.value+1
                    total_beds = total_beds + prov_beds
                    load_progress.value=load_progress.value+1

                    if df1.empty == False:
                        confirmed = df1["Confirmed"].iloc[0]
                        recovered = df1["Recovered"].iloc[0]
                        active = df1["Active"].iloc[0]

                        confirmed_total = confirmed_total + confirmed        
                        recovered_total = recovered_total + recovered
                        active_total = active_total + active

                        N = 40369 #prov's population
                        interact_num_of_death(N, recovered, confirmed, active, prov_beds)        

                with out1: #Canada
                    df_pop = df_world_population[df_world_population['Location']==get_population_country_name(chosen_country)]
                    load_progress.value=load_progress.value+1
                    N = df_pop['PopTotal'].iloc[0]

                    interact_num_of_death(N.astype(int), recovered_total, confirmed_total, active_total, total_beds)  
    except Exception as e:
        print('78')                    

In [ ]:
def display_all_graphs(Country, Date): 
    if Country=='Select Country' or Date=='Select Date':
        return
    
    try:
        global load_progress
        global today    
        global df_covid
        global chosen_country

        chosen_country=Country

        if chosen_country=='Canada':
            load_progress=widgets.IntProgress(value=0,
                                              min=0,
                                              max=131,
                                              step=1,                                      
                                              bar_style='info',
                                              orientation='horizontal')
        else:
            load_progress=widgets.IntProgress(value=0,
                                              min=0,
                                              max=14,
                                              step=1,                                      
                                              bar_style='info',
                                              orientation='horizontal')  

        lbl_wait = widgets.Label(value='Fetching and processing ' + Date + ' data for ' + chosen_country + ':')
        display(HBox([lbl_wait, load_progress]))

        today = Date
        url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + today + ".csv"

        if not exists(url): #check if no file with yesterday's date, get file from 2 days ago
            today = date.today() + timedelta(-2)
            today = today.strftime("%m-%d-%Y")
            url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + today + ".csv"

        df = pd.read_csv(url)   
        load_progress.value=load_progress.value+1
        df = df.replace(np.nan, 0)
        load_progress.value=load_progress.value+1
        df = df.drop(df[(df.Province_State == 'Diamond Princess') | 
                        (df.Province_State == 'Grand Princess')
                        ].index)
        load_progress.value=load_progress.value+1
        df = df.drop(df[(df.Country_Region == 'Diamond Princess') |
                           (df.Country_Region == 'South Sudan') |
                           (df.Country_Region == 'Congo (Brazzaville)') |
                           (df.Country_Region == 'Congo (Kinshasa)') |
                           (df.Country_Region == "Cote d'Ivoire") |
                           (df.Country_Region == 'Holy See') |
                           (df.Country_Region == 'Kosovo') |
                           (df.Country_Region == 'Liechtenstein') |
                           (df.Country_Region == 'MS Zaandam') |
                           (df.Country_Region == 'West Bank and Gaza')                        
                       ].index)
        load_progress.value=load_progress.value+1
        df_covid = df

        display_graphs(df)
        clear_output()
        display(output)
        display(tabs)   

        lbl_buffer=widgets.Label(value=' ')
        for space_ctr in range(0, 3):        
            display(lbl_buffer)      
    except Exception as e:
        print(e)
        print('79')            

In [ ]:
widgets.HTML(
            value="<i><h2>" +
                    "y = (S<sub>m</sub><sup>(m)</sup>, "+
                    "E<sub>m</sub><sup>(m)</sup>, "+
                    "A<sub>m</sub><sup>(m)</sup>, "+
                    "I<sub>m</sub><sup>(m)</sup>, "+
                    "R<sub>m</sub><sup>(m)</sup>,<br/> "+
                    "&nbsp;&nbsp;&nbsp;&nbsp;S<sub>m</sub><sup>(s)</sup>, "+
                    "E<sub>m</sub><sup>(s)</sup>, "+
                    "A<sub>m</sub><sup>(s)</sup>, "+
                    "I<sub>m</sub><sup>(s)</sup>, "+
                    "R<sub>m</sub><sup>(s)</sup>,<br/>"+
                    "&nbsp;&nbsp;&nbsp;&nbsp;S<sub>s</sub><sup>(s)</sup>, "+
                    "E<sub>s</sub><sup>(s)</sup>, "+
                    "A<sub>s</sub><sup>(s)</sup>, "+
                    "I<sub>s</sub><sup>(s)</sup>, "+
                    "ICU<sub>s</sub><sup>(s)</sup>, "+
                    "R<sub>s</sub><sup>(s)</sup>,<br/>"+
                    "&nbsp;&nbsp;&nbsp;&nbsp;S<sub>s</sub><sup>(m)</sup>, "+
                    "E<sub>s</sub><sup>(m)</sup>, "+
                    "A<sub>s</sub><sup>(m)</sup>, "+
                    "I<sub>s</sub><sup>(m)</sup>, "+
                    "ICU<sub>s</sub><sup>(m)</sup>, "+
                    "R<sub>s</sub><sup>(m)</sup>,<br/>"+
                    "&nbsp;&nbsp;&nbsp;&nbsp;D)</h2></i>"
            ,placeholder='Some HTML'    
)

In [ ]:
get_country_hospital_beds()

In [ ]:
get_covid_country()

In [ ]:
get_covid_dates()

In [ ]:
get_world_population()

In [ ]:
## Capacity
I_max = 10000.   # Healthcare capacity

# Hypothesis
#p = 0.995        # Proportion of people with mild symptoms
delta_r = 0.1    # to be thought of how much released people have lowered their exits from home
delta_c = 0.6    # to be thought of how much people in lockdown have lowered their exits from home

In [ ]:
# Parameters for the distribution of people with severe symptoms
alpha_p = 4.5
beta_p = 2
# Parameters for the distribution of people with mild symptoms
alpha_n = 2
beta_n = 4.5

In [ ]:
mild = np.genfromtxt('data/mild_a.csv', delimiter=',')
severe = np.genfromtxt('data/severe_a.csv', delimiter=',')

In [ ]:
date = datetime.date(2020,3,22) #this is the oldest date where Canada was first recorded

In [ ]:
widgets.HTML(value='<h3>Click on a country marker to see latest summary.</h3>', placeholder='map_instruction')

In [ ]:
m_today = date.today() + timedelta(-1)
m_today = m_today.strftime("%m-%d-%Y")

m_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + m_today + ".csv"

if not exists(m_url): #check if no file with yesterday's date, get file from 2 days ago
    m_today = date.today() + timedelta(-2)
    m_today = m_today.strftime("%m-%d-%Y")
    m_url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + m_today + ".csv"

#x=widgets.HTML(value='here1 '+m_url, placeholder='map_url')
#display(x)

try:
    m_df = pd.read_csv(m_url)
    #x=widgets.HTML(value='good1', placeholder='map_url')
    #display(x)
except Exception as e:
    x=widgets.HTML(value='err1 '+e, placeholder='map_error')
    display(x)

try:
    m_df = m_df.drop(m_df[(m_df.Province_State == 'Diamond Princess') | (m_df.Province_State == 'Grand Princess')].index)
    #x=widgets.HTML(value='good2', placeholder='map_url')
    #display(x)
except Exception as e:
    x=widgets.HTML(value='err2 '+e, placeholder='map_error')
    display(x)    

try:
    m_df = m_df.drop(m_df[(m_df.Country_Region == 'South Sudan')].index)
    #x=widgets.HTML(value='good3', placeholder='map_url')
    #display(x)    
except Exception as e:
    x=widgets.HTML(value='err3 '+e, placeholder='map_error')
    display(x)

In [ ]:
#Creating a base map
df_country_latlong = pd.read_csv('countries_lat_long.csv')
m = folium.Map(tiles="Stamen Terrain", control_scale=True, height='85%', width='90%', max_bounds=True)
minimap = plugins.MiniMap(toggle_display=True)

plugins.Fullscreen(position='topleft').add_to(m)

if m_df.empty==False:
    for country_code, lat, lon, name in zip(df_country_latlong['country'],df_country_latlong['latitude'],df_country_latlong['longitude'],df_country_latlong['name']):
        #Creating the marker
        m_df_country=m_df[m_df['Country_Region']==name]
        recovered=m_df_country['Recovered'].sum()
        infected=m_df_country['Confirmed'].sum()
        icu=m_df_country['Active'].sum()
        deaths=m_df_country['Deaths'].sum()
        inc_rate=m_df_country['Incidence_Rate'].mean() if 'Incidence_Rate' in m_df_country.columns else 0
        case_fatal_ratio=m_df_country['Case-Fatality_Ratio'].mean() if 'Case-Fatality_Ratio' in m_df_country.columns else 0

        folium.Marker(
                        #Coordinate of the country
                        location=[lat, lon],
                        #The popup that show up if click the marker
                        popup=folium.Popup(show_stats(name, m_today, recovered, infected, icu, deaths, inc_rate, case_fatal_ratio, country_code), max_width=800, min_width=200)
                    ).add_to(m)
else:
    no_map_function=widgets.HTML(value='<h3>We apologize. Map functionalities currently unavailable.</h3>', placeholder='no_map_function')
    display(no_map_function)
    
m.add_child(minimap)    
m

In [ ]:
call_general_graphs()

In [ ]:
#m1=interactive(display_all_graphs, 
#                Country=[country for country in df_selection_country['Country_Region'].unique()],
#                Date=[date for date in df_date['Dates'].unique()])
lbl_selection_instruction=widgets.HTML(value='<h3>Select country and date to see simulation.</h3><i>Please check your network for this site to function.</i>', placeholder='lbl_selection_instruction')
#display(lbl_selection_instruction)
#display(m1)

Country=widgets.Dropdown(options=[country for country in df_selection_country['Country_Region'].unique()])
Date=widgets.Dropdown(options=[date for date in df_date['Dates'].unique()])

ui=widgets.HBox([Country, Date])
out=widgets.interactive_output(display_all_graphs, {'Country': Country, 'Date': Date})

display(lbl_selection_instruction)
display(ui ,out)

In [ ]:
list_widgets  = [
                widgets.VBox([
                                widgets.Label(value='Visualizations and Simulations for COVID-Related Analytics'),
                                widgets.Label(value=' '),                    
                                widgets.Label(value='For:'),
                                widgets.Label(value='Prof. Anton Ovchinnikov, Prof. Mikhail Nediak'),
                                widgets.Label(value='Capstone, Master of Management in Artificial Intelligence'),
                                widgets.Label(value='Smith School of Business'),
                                widgets.Label(value="Queen's University"),
                                widgets.Label(value=' '),
                                widgets.Label(value='Based on the paper:'),
                                widgets.Label(value='Personalised COVID-19 Isolation and Exit Policies Using Epidemic Simulations With Clinical Risk Predictions '),
                                widgets.Label(value='by: T. Evgeniou, M. Fekom, A. Ovchinnikov, R. Porcher, C. Pouchol, N. Vayatis'),
                                widgets.Label(value='May 2020'),
                                widgets.Label(value=' '),
                                widgets.Label(value='Special thanks to:'),
                                widgets.Label(value='Camille Pouchol (seed code, France)'),
                                widgets.Label(value='Harshdeep Singh (technical advisor)'),
                                widgets.Label(value=' ') ,                   
                                widgets.Label(value='MMAI students:'), 
                                widgets.Label(value='Francis Bello (il.francis.bello@gmail.com), Lingfeng (Randy) Liao (randyliao12@gmail.com), Imad Jiwadi (ijawadi@gmail.com)'), 
                                widgets.Label(value='2020')
                             ])
                ]

accordion = widgets.Accordion(children=list_widgets, selected_index=None)
accordion.set_title(0, 'About')
accordion